<a href="https://colab.research.google.com/github/matt14e/StitchAI/blob/main/Stitchmodel3_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install lightning torchvision pyembroidery pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.0/819.0 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Edit here if folders are moved
DATA_ROOT = "/content/drive/MyDrive/Embroidery Files"
IMG_DIR   = f"{DATA_ROOT}/PNG_image_files"   # PNGs
DST_DIR   = f"{DATA_ROOT}/DST_digitized_files"   # DSTs

In [4]:
#sanity check the folders
import pathlib, textwrap

print("📷  sample PNG files:")
for p in list(pathlib.Path(IMG_DIR).glob("*.[pP][nN][gG]"))[:5]:
    print("   ", p.name)

print("\n🧵  sample DST files:")
for p in list(pathlib.Path(DST_DIR).glob("*.[dD][sS][tT]"))[:5]:
    print("   ", p.name)


📷  sample PNG files:
    Cod_Fish_Logo.PNG
    MLPP_LOGO.PNG
    LF_LOGO.PNG
    Andrews_Logo.PNG
    Royal_Logo.PNG

🧵  sample DST files:
    Elite.DST
    Lehigh.DST
    Moon-Walkaz-Text.DST
    Kamp.DST
    Nike.DST


In [5]:
%%writefile dataset.py
from pathlib import Path
from PIL import Image
import torch
from torch.utils.data import Dataset
from pyembroidery import read

# --- file patterns -----------------------------------------------------------
IMG_EXTS = ("*.png", "*.PNG", "*.jpg", "*.JPG")
DST_EXTS = ("*.dst", "*.DST")

class EmbroDataset(Dataset):
    """
    Yields (image_tensor, stitch_tensor) pairs.

    image_tensor  : C × H × W   float32 in [0, 1]
    stitch_tensor : L × 3       (Δx, Δy, flag)  float32
    """
    def __init__(self, img_dir, dst_dir, transform=None, max_len=4096):
        img_dir, dst_dir = Path(img_dir), Path(dst_dir)

        # -------- gather every image and dst file into dicts ---------------
        img_files = {}
        for pat in IMG_EXTS:
            for p in img_dir.glob(pat):
                img_files[p.stem] = p

        dst_files = {}
        for pat in DST_EXTS:
            for d in dst_dir.glob(pat):
                dst_files[d.stem] = d

        # -------- keep only names that exist in *both* dicts --------------
        self.common_names = sorted(img_files.keys() & dst_files.keys())
        if not self.common_names:
            raise RuntimeError("No matching (image, DST) pairs found!")

        self.img_files  = img_files
        self.dst_files  = dst_files
        self.transform  = transform
        self.max_len    = max_len

    # ------------------------------------------------------------------------
    def __len__(self):
        return len(self.common_names)

    def __getitem__(self, idx):
        name = self.common_names[idx]

        # load & optionally transform image
        img_path = self.img_files[name]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)

        # load DST stitches
        dst_path  = self.dst_files[name]
        pattern   = read(str(dst_path))
        stitches  = torch.tensor(pattern.stitches,
                                 dtype=torch.float32)[: self.max_len]

        return img, stitches


Writing dataset.py


In [ ]:
#dont run

%%writefile dataset.py
from pathlib import Path
from PIL import Image
import torch
from torch.utils.data import Dataset
from pyembroidery import read

IMG_EXTS = ("*.png", "*.PNG", "*.jpg", "*.JPG")
DST_EXTS = ("*.dst", "*.DST")

class EmbroDataset(Dataset):
    """
    (image_tensor, stitch_tensor) pairs
    image_tensor  : C×H×W  float32 [0,1]
    stitch_tensor : L×3    (Δx, Δy, flag)
    """
    def __init__(self, img_dir, dst_dir, transform=None, max_len=4096):
        self.img_paths = []
        for pat in IMG_EXTS:
            self.img_paths.extend(Path(img_dir).glob(pat))
        self.img_paths = sorted(self.img_paths)

        self.dst_dir   = Path(dst_dir)
        self.transform = transform
        self.max_len   = max_len

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)

        stem = img_path.stem
        for ext in DST_EXTS:
            cand = self.dst_dir / f"{stem}{ext[1:]}"   # '*.dst' ➜ '.dst'
            if cand.exists():
                dst_path = cand
                break
        else:
            raise FileNotFoundError(f"No DST match for {stem}")

        pattern  = read(str(dst_path))
        stitches = torch.tensor(pattern.stitches,
                                dtype=torch.float32)[: self.max_len]
        return img, stitches


In [6]:
%%writefile model.py
import torch, torch.nn as nn

class EmbroNet(nn.Module):
    """Tiny CNN encoder + GRU decoder baseline."""
    def __init__(self, hidden=256):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, 2, 1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, 2, 1), nn.ReLU(),
            nn.Conv2d(64,128, 3, 2, 1), nn.ReLU(),
            nn.AdaptiveAvgPool2d(1)          # 128×1×1
        )
        self.enc_fc  = nn.Linear(128, hidden)

        self.gru     = nn.GRU(3, hidden, num_layers=2, batch_first=True)
        self.dec_fc  = nn.Linear(hidden, 3)

    def forward(self, img, prev_cmds):
        B = img.size(0)
        h0 = self.encoder(img).view(B, -1)      # B×128
        h0 = torch.tanh(self.enc_fc(h0)).unsqueeze(0).repeat(2, 1, 1)
        out, _ = self.gru(prev_cmds, h0)        # B×L×H
        return self.dec_fc(out)                 # B×L×3


Writing model.py


In [7]:
import torch
import lightning as L
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor, Resize
from dataset import EmbroDataset
from model   import EmbroNet

# ---------- 1. custom collate_fn ---------------------------------
def pad_collate(batch):
    """
    batch = list of (img_tensor, seq_tensor) pairs
    Returns:
        imgs   : B×3×128×128
        tgt    : B×Lmax×3  (padded with 0s)
        lens   : list[int] (original sequence lengths)
    """
    imgs, seqs = zip(*batch)
    imgs = torch.stack(imgs)                   # all same size

    lens    = [s.size(0) for s in seqs]
    Lmax    = max(lens)
    padded  = torch.zeros(len(seqs), Lmax, 3)  # default zeros = padding
    for i, s in enumerate(seqs):
        padded[i, : lens[i], :] = s
    return imgs, padded, torch.tensor(lens)

# ---------- 2. LightningModule -----------------------------------
class LitModule(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.net = EmbroNet()
        self.loss_fn = torch.nn.MSELoss(reduction="mean")

    def forward(self, img, prev_cmds):
        return self.net(img, prev_cmds)

    def training_step(self, batch, _):
        img, tgt, lens = batch                 # tgt: B×Lmax×3
        pred = self(img, tgt[:, :-1])

        # compute loss mask-aware
        loss = 0.0
        for i, L in enumerate(lens):
            loss += self.loss_fn(pred[i, : L-1], tgt[i, 1 : L])  # valid region
        loss = loss / len(lens)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), 1e-3)

# ---------- 3. dataset & dataloader ------------------------------
tfms = Compose([Resize((128,128)), ToTensor()])
ds   = EmbroDataset(IMG_DIR, DST_DIR, tfms)
print("Dataset length:", len(ds))          # should be > 0

dl   = DataLoader(ds,
                  batch_size=8,
                  shuffle=True,
                  num_workers=0,            # change later if you like
                  collate_fn=pad_collate)   # ← custom collate!

# ---------- 4. train ---------------------------------------------
trainer = L.Trainer(fast_dev_run=True)
#trainer = L.Trainer(max_epochs=10,
                    #precision="16-mixed",
                    #accelerator="auto")
trainer.fit(LitModule(), dl)


INFO: Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:lightning.pytorch.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
INFO:lightning.pytorch.utilities.rank_zero:Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkp

Dataset length: 161


INFO: 
  | Name    | Type     | Params | Mode 
---------------------------------------------
0 | net     | EmbroNet | 722 K  | train
1 | loss_fn | MSELoss  | 0      | train
---------------------------------------------
722 K     Trainable params
0         Non-trainable params
722 K     Total params
2.889     Total estimated model params size (MB)
13        Modules in train mode
0         Modules in eval mode
INFO:lightning.pytorch.callbacks.model_summary:
  | Name    | Type     | Params | Mode 
---------------------------------------------
0 | net     | EmbroNet | 722 K  | train
1 | loss_fn | MSELoss  | 0      | train
---------------------------------------------
722 K     Trainable params
0         Non-trainable params
722 K     Total params
2.889     Total estimated model params size (MB)
13        Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_steps=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1` reached.


In [ ]:
#Using the Model
# ================================================================
# 🔄  PNG  ➜  trained model  ➜  DST
# ================================================================
import torch, pathlib
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from pyembroidery import EmbPattern, write
from model import EmbroNet            # your network definition

# --- 1. paths ----------------------------------------------------
CKPT_PATH = "/content/lightning_logs/version_0/checkpoints/epoch=9-step=199.ckpt"
PNG_PATH  = "/content/drive/MyDrive/Embroidery Files/pngstart/MLPP_LOGO.PNG"     # <-- put your PNG here
OUT_PATH  = "/content/drive/MyDrive/Embroidery Files/dstend/finisheddst.DST"

# --- 2. load network --------------------------------------------
net = EmbroNet().eval()
state = torch.load(CKPT_PATH, map_location="cpu")
net.load_state_dict(state["state_dict"] if "state_dict" in state else state)
print("✓ checkpoint loaded")

# --- 3. preprocess PNG ------------------------------------------
tfms = Compose([Resize((128,128)), ToTensor()])
img  = tfms(Image.open(PNG_PATH).convert("RGB")).unsqueeze(0)  # 1×3×128×128

# --- 4. autoregressive inference -------------------------------
MAX_LEN = 4096           # stop if you hit this many stitches
with torch.no_grad():
    seq = torch.zeros(1, 1, 3)          # start token
    for _ in range(MAX_LEN):
        pred = net(img, seq)            # 1×L×3
        next_cmd = pred[:, -1:, :]
        seq = torch.cat([seq, next_cmd], dim=1)

pred_seq = seq.squeeze(0)[1:]           # drop the start token

# --- 5. (optional) rescale Δx,Δy if you trained un‑normalised ----
SCALE = 7.0           # ≈ (ground‑truth Δ)/ (predicted Δ)  ← use your ratio
pred_seq[:, :2] *= SCALE

# --- 6. build an EmbPattern -------------------------------------
pat = EmbPattern()
x = y = 0.0
for dx, dy, flag in pred_seq.cpu().numpy():
    x += dx;  y += dy
    pat.stitch(x, y)           # you can use flag to insert JUMP/TRIM later
pat.end()

# --- 7. write DST -----------------------------------------------
write(pat, OUT_PATH)
print("✓ saved", OUT_PATH)


In [ ]:
img, tgt = next(iter(dl))
net = trainer.model.net.to(img.device)
with torch.no_grad():
    pred = net(img, tgt[:, :-1])
print("GT  :", tgt[0, :5])
print("Pred:", pred[0, :5])